In [1]:
# Importing basic python libraries

import os 
import time
import numpy as np
import math
import threading
import sys
import time
from pynq_dpu import DpuOverlay
from scipy.io import loadmat
from queue import Queue

In [2]:
# Importing DPU Overlay blocks and .xmodel
overlay = DpuOverlay("dpu.bit")
overlay.load_model("model.xmodel")

In [3]:
dataset = loadmat('test_data')
raw_data = dataset['data']
label = dataset['label']
print("Shape of input data is {}".format(raw_data.shape))

Shape of input data is (6645, 8, 40)


In [ ]:
mu = raw_data.mean(axis=2)
std = raw_data.std(axis=2)
data = np.zeros(raw_data.shape)
for indx, sample in enumerate(raw_data):
    data[indx]  = ((raw_data[indx].transpose() - mu[indx]) / std[indx]).transpose()


In [4]:
data = np.expand_dims(raw_data, axis=3)
data.shape

(6645, 8, 40, 1)

In [5]:
dpu = overlay.runner

inputTensors = dpu.get_input_tensors()
outputTensors = dpu.get_output_tensors()

shapeIn = tuple(inputTensors[0].dims)
shapeOut = tuple(outputTensors[0].dims)
outputSize = int(outputTensors[0].get_data_size() / shapeIn[0])

softmax = np.empty(outputSize)

In [6]:
output_data = [np.empty(shapeOut, dtype=np.float32, order="C")]
input_data = [np.empty(shapeIn, dtype=np.float32, order="C")]
image = input_data[0]

In [7]:
def calculate_softmax(data):
    result = np.exp(data)
    return result

In [8]:
num_samples  = 10
for i in range(num_samples):
    image[0,...] = data[i]
    job_id = dpu.execute_async(input_data, output_data)
    dpu.wait(job_id)
    temp = [j.reshape(1, outputSize) for j in output_data]
    softmax = calculate_softmax(temp[0][0])
    prediction = softmax.argmax()
    gnd_truth = label[i]
    
    print('Prediction: {} && True Label {}'.format(prediction, gnd_truth))

Prediction: 0 && True Label [0.]
Prediction: 4 && True Label [4.]
Prediction: 0 && True Label [0.]
Prediction: 5 && True Label [5.]
Prediction: 3 && True Label [3.]
Prediction: 2 && True Label [2.]
Prediction: 6 && True Label [6.]
Prediction: 4 && True Label [4.]
Prediction: 5 && True Label [5.]
Prediction: 6 && True Label [6.]


In [9]:
total = data.shape[0]
predictions = np.empty_like(label)
print("Gesture Recognition to Estimate Latency")

start = time.time()
for i in range(total):
    image[0,...] = data[i]
    job_id = dpu.execute_async(input_data, output_data)
    dpu.wait(job_id)
    temp = [j.reshape(1, outputSize) for j in output_data]
    softmax = calculate_softmax(temp[0][0])
    predictions[i] = softmax.argmax()

stop = time.time()
correct = np.sum(predictions==label)
execution_time = stop-start
print("Overall accuracy: {}".format(correct/total))
print("Latency: {:.4f}s".format(execution_time / total))

Gesture Recognition to Estimate Latency
Overall accuracy: 0.9232505643340858
Latency: 0.0003s


In [10]:
del overlay
del dpu